我使用了flyai这个竞赛平台进行文本分类，我使用的是用tensorflow实现的fasttext网络

算法的大致框架如下，如下部分是构建fasttext网络，结构很简单，只需要一层简单的网络层

with tf.variable_scope('embedding'):
    # 标准正态分布初始化
    input_embedding = tf.Variable(
        tf.truncated_normal(shape=[vocab_size, embedding_dim], stddev=0.1), name='encoder_embedding')

with tf.name_scope("fasttext"):
    
    x_input_embedded = tf.nn.embedding_lookup(input_embedding, input_x)
    # conv = tf.layers.conv1d(x_input_embedded, num_filters, kernel_size, name='conv')
    # # global max pooling layer
    # pooling = tf.reduce_max(conv, reduction_indices=[1])
    sentence_embedd = tf.reduce_mean(x_input_embedded, axis=1)
    w = tf.get_variable('W', [embedding_dim, numclass])
    b = tf.get_variable('b', [numclass])
    logits = tf.matmul(sentence_embedd, w) + b

以下是代码的损失函数精度，以及训练过程

l2_reg_lambda = 0.0
with tf.name_scope("score"):
    # 全连接层，后面接dropout以及relu激活
    # fc = tf.layers.dense(pooling, hidden_dim, name='fc1')
    # fc = tf.contrib.layers.dropout(fc, keep_prob)
    # fc = tf.nn.relu(fc)
    # loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=input_y)
    # L2_loss = tf.nn.l2_loss(w)
    # L2_loss += tf.nn.l2_loss(b)
    # loss += L2_loss * l2_reg_lambda


    # # 分类器
    # logits = tf.layers.dense(fc, numclass, name='fc2')

    y_pred_cls = tf.argmax(tf.nn.softmax(logits), 1, name='y_pred')  # 预测类别

with tf.name_scope("optimize"):
    # 将label进行onehot转化
    one_hot_labels = tf.one_hot(input_y, depth=numclass, dtype=tf.float32)
    # 损失函数，交叉熵
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=one_hot_labels)
    loss = tf.reduce_mean(cross_entropy)
    # 优化器
    train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

with tf.name_scope("accuracy"):
    # 准确率
    correct_pred = tf.equal(tf.argmax(one_hot_labels, 1), y_pred_cls)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='acc')

with tf.name_scope("summary"):
    tf.summary.scalar("loss", loss)
    tf.summary.scalar("accuracy", accuracy)
    merged_summary = tf.summary.merge_all()

best_score = 0
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter(LOG_PATH, sess.graph)

    # dataset.get_step() 获取数据的总迭代次数
    for step in range(dataset.get_step()):
        x_train, y_train = dataset.next_train_batch()
        # x_val, y_val = dataset.next_validation_batch()

        fetches = [loss, accuracy, train_op]
        feed_dict = {input_x: x_train, input_y: y_train, keep_prob: 0.9}
        loss_, accuracy_, _ = sess.run(fetches, feed_dict=feed_dict)

        summary = sess.run(merged_summary, feed_dict=feed_dict)
        train_writer.add_summary(summary, step)

        cur_step = str(step + 1) + "/" + str(dataset.get_step())
        print('The Current step per total: {} | The Current loss: {} | The Current ACC: {}'.
              format(cur_step, loss_, accuracy_))
        if step % 100 == 0:
            model.save_model(sess, MODEL_PATH, overwrite=True)

我们可以看看训练过程的日志，分类比其他的网络都要快，效果也不错

The Current step per total: 459/500 | The Current loss: 0.1529412865638733 | The Current ACC: 1.0
The Current step per total: 460/500 | The Current loss: 0.5357775092124939 | The Current ACC: 0.800000011920929
The Current step per total: 461/500 | The Current loss: 0.3405835032463074 | The Current ACC: 0.8999999761581421
The Current step per total: 462/500 | The Current loss: 0.15472419559955597 | The Current ACC: 1.0
The Current step per total: 463/500 | The Current loss: 0.15399295091629028 | The Current ACC: 1.0
The Current step per total: 464/500 | The Current loss: 0.303913414478302 | The Current ACC: 0.800000011920929
The Current step per total: 465/500 | The Current loss: 0.19542500376701355 | The Current ACC: 0.8999999761581421
The Current step per total: 466/500 | The Current loss: 0.25718802213668823 | The Current ACC: 0.8999999761581421
The Current step per total: 467/500 | The Current loss: 0.41457900404930115 | The Current ACC: 0.8999999761581421
The Current step per total: 468/500 | The Current loss: 0.590376079082489 | The Current ACC: 0.800000011920929
The Current step per total: 469/500 | The Current loss: 0.1942882090806961 | The Current ACC: 1.0
The Current step per total: 470/500 | The Current loss: 0.14795874059200287 | The Current ACC: 1.0
The Current step per total: 471/500 | The Current loss: 0.5234373807907104 | The Current ACC: 0.800000011920929
The Current step per total: 472/500 | The Current loss: 0.33145222067832947 | The Current ACC: 0.8999999761581421
The Current step per total: 473/500 | The Current loss: 0.1468111276626587 | The Current ACC: 1.0
The Current step per total: 474/500 | The Current loss: 0.14611704647541046 | The Current ACC: 1.0
The Current step per total: 475/500 | The Current loss: 0.29498711228370667 | The Current ACC: 0.800000011920929
The Current step per total: 476/500 | The Current loss: 0.18875113129615784 | The Current ACC: 0.8999999761581421
The Current step per total: 477/500 | The Current loss: 0.24813206493854523 | The Current ACC: 0.8999999761581421
The Current step per total: 478/500 | The Current loss: 0.4078347682952881 | The Current ACC: 0.8999999761581421
The Current step per total: 479/500 | The Current loss: 0.5791069865226746 | The Current ACC: 0.800000011920929
The Current step per total: 480/500 | The Current loss: 0.18591205775737762 | The Current ACC: 1.0
The Current step per total: 481/500 | The Current loss: 0.14316657185554504 | The Current ACC: 1.0
The Current step per total: 482/500 | The Current loss: 0.511402428150177 | The Current ACC: 0.800000011920929
The Current step per total: 483/500 | The Current loss: 0.3227069079875946 | The Current ACC: 0.8999999761581421
The Current step per total: 484/500 | The Current loss: 0.13929709792137146 | The Current ACC: 1.0
The Current step per total: 485/500 | The Current loss: 0.13863947987556458 | The Current ACC: 1.0
The Current step per total: 486/500 | The Current loss: 0.28622403740882874 | The Current ACC: 0.800000011920929
The Current step per total: 487/500 | The Current loss: 0.18225839734077454 | The Current ACC: 0.8999999761581421
The Current step per total: 488/500 | The Current loss: 0.23940058052539825 | The Current ACC: 0.8999999761581421
The Current step per total: 489/500 | The Current loss: 0.40118408203125 | The Current ACC: 0.8999999761581421
The Current step per total: 490/500 | The Current loss: 0.5678361654281616 | The Current ACC: 0.800000011920929
The Current step per total: 491/500 | The Current loss: 0.17787066102027893 | The Current ACC: 1.0
The Current step per total: 492/500 | The Current loss: 0.13856475055217743 | The Current ACC: 1.0
The Current step per total: 493/500 | The Current loss: 0.4996697008609772 | The Current ACC: 0.800000011920929
The Current step per total: 494/500 | The Current loss: 0.31433865427970886 | The Current ACC: 0.8999999761581421
The Current step per total: 495/500 | The Current loss: 0.13217487931251526 | The Current ACC: 1.0
The Current step per total: 496/500 | The Current loss: 0.1315530240535736 | The Current ACC: 1.0
The Current step per total: 497/500 | The Current loss: 0.27763742208480835 | The Current ACC: 0.800000011920929
The Current step per total: 498/500 | The Current loss: 0.1759549379348755 | The Current ACC: 0.8999999761581421
The Current step per total: 499/500 | The Current loss: 0.23099246621131897 | The Current ACC: 0.8999999761581421
The Current step per total: 500/500 | The Current loss: 0.394628643989563 | The Current ACC: 0.8999999761581421
